# Stock Prediction Model

## Dependencies

### Library Installation (if needed)

In [30]:
# Install libraries the first time
#! pip install -U yfinance pandas pathlib numpy 

### Importing Required Libraries

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

## Data Preparation

In [32]:
symbols = pd.read_csv('../data/simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
symbols

array(['MATAS.CO', 'TRIFOR.CO', 'QQ.L', 'RNMBY', 'SAABF', 'BCKIY',
       'BAESY', 'IVSO.ST', 'NSKFF', 'GMAB', 'GN.CO', 'NVDA', 'LLY',
       'DANSKE.CO', 'CARL-B.CO', 'MAERSK-B.CO', 'RBREW.CO', 'ISS.CO',
       'DSV.CO', 'SCHO.CO', 'NETC.CO', 'JYSK.CO', 'ABBN.SW', 'TER',
       'PARKEN.CO', 'NFLX', 'TRMD-A.CO', 'STG.CO', 'NOVO-B.CO', 'EQNR',
       'NKT.CO', 'NSIS-B.CO', 'KCC.OL'], dtype=object)

#### Download and Process Historical Data

In [ ]:
print_errors=True

df = pd.DataFrame()
for symbol in tqdm(symbols):
    ticker_df = pd.DataFrame()
    ticker = yf.Ticker(symbol)
    earning_dates = ticker.cash_flow.columns.tolist()
    price_data = yf.download(symbol, period='10y', rounding=False, progress=False)
    cash_flow = ticker.cash_flow
    cash_flow_columns = cash_flow[earning_dates[0]].keys().tolist()
    balance_sheet = ticker.balance_sheet
    balance_sheet_columns = balance_sheet[earning_dates[0]].keys().tolist()
    income_statement = ticker.income_stmt
    income_statement_columns = income_statement[earning_dates[0]].keys().tolist()

    for earning_date in earning_dates:
        try:
            earning_date_data = {'Ticker': symbol, 'Date': earning_date}
        
            got_price = False
            day_offset = 0
            while(got_price==False and day_offset > -6):
                try:
                    earning_date_data['1 Year Future Change'] = float(price_data.loc[earning_date + pd.Timedelta(days=day_offset+365,), ('Close', symbol)]) / float(price_data.loc[earning_date + pd.Timedelta(days=day_offset), ('Close', symbol)]) - 1
                    got_price = True
                except Exception:
                    day_offset += -1

            for column in cash_flow_columns:
                earning_date_data[column] = cash_flow[earning_date][column]

            for column in balance_sheet_columns:
                earning_date_data[column] = balance_sheet[earning_date][column]

            for column in income_statement_columns:
                earning_date_data[column] = income_statement[earning_date][column]
                
        except Exception as error:
            if print_errors:
                print(f'Error for {symbol}: {error}')
        
        if got_price:
            ticker_df = pd.concat([ticker_df, pd.DataFrame([earning_date_data])], ignore_index=True)

    if ticker_df.columns[ticker_df.isna().all()].tolist():
        continue
    imputer = SimpleImputer()
    wanted_columns = ticker_df.columns.drop(['Ticker', 'Date', '1 Year Future Change'])
    for column in wanted_columns:
        ticker_df[column] = imputer.fit_transform(ticker_df[[column]])

    df = pd.concat([df, ticker_df], ignore_index=True)

df.to_csv('../data/earnings_data.csv', index=False)

  0%|          | 0/33 [00:00<?, ?it/s]

  3%|▎         | 1/33 [00:00<00:31,  1.01it/s]c:\Users\Gamer\miniconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Other Non Current Assets']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
  9%|▉         | 3/33 [00:03<00:36,  1.21s/it]c:\Users\Gamer\miniconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Tradeand Other Payables Non Current']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
 18%|█▊        | 6/33 [00:08<00:40,  1.50s/it]c:\Users\Gamer\miniconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Cash Equivalents']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
c:\Users\Gamer\miniconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning:

Error for NSKFF: Timestamp('2020-12-31 00:00:00')


 30%|███       | 10/33 [00:14<00:35,  1.52s/it]

Error for GN.CO: Timestamp('2020-12-31 00:00:00')


...

 85%|████████▍ | 28/33 [00:43<00:08,  1.67s/it]

Error for NOVO-B.CO: Timestamp('2020-12-31 00:00:00')


 88%|████████▊ | 29/33 [00:45<00:06,  1.70s/it]c:\Users\Gamer\miniconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Other Non Current Assets']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
 91%|█████████ | 30/33 [00:47<00:04,  1.66s/it]

Error for NKT.CO: Timestamp('2020-12-31 00:00:00')


 94%|█████████▍| 31/33 [00:48<00:03,  1.67s/it]c:\Users\Gamer\miniconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Net Other Financing Charges']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
 97%|█████████▋| 32/33 [00:50<00:01,  1.72s/it]c:\Users\Gamer\miniconda3\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['Other Equity Interest']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(
100%|██████████| 33/33 [00:52<00:00,  1.58s/it]


### Short visualisation

In [34]:
df

,Ticker,Date,1 Year Future Change,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Capital Expenditure,End Cash Position,Beginning Cash Position,...,Non Current Accrued Expenses,Earnings From Equity Interest Net Of Tax,Depreciation Amortization Depletion Income Statement,Amortization Of Intangibles Income Statement,Preferred Stock,Non Current Note Receivables,Line Of Credit,Commercial Paper,Current Notes Payable,Preferred Stock Dividends
0,MATAS.CO,2024-03-31,0.153992,2.390000e+08,-2.100000e+07,-1.890000e+08,1.121000e+09,-4.060000e+08,1.310000e+08,3.700000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MATAS.CO,2023-03-31,0.470333,4.240000e+08,0.000000e+00,-1.260000e+08,0.000000e+00,-2.540000e+08,3.700000e+07,2.800000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MATAS.CO,2022-03-31,-0.103056,3.271000e+08,-7.510000e+07,-7.084000e+08,7.635000e+08,-1.834000e+08,2.820000e+07,4.070000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MATAS.CO,2021-03-31,0.179800,8.036000e+08,0.000000e+00,-7.250000e+08,0.000000e+00,-1.484000e+08,4.070000e+07,1.066000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,QQ.L,2024-03-31,0.075779,1.468000e+08,-1.710000e+07,0.000000e+00,0.000000e+00,-9.630000e+07,2.310000e+08,1.512000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,NOVO-B.CO,2020-12-31,0.756239,6.058700e+10,-2.448567e+10,-7.259667e+09,1.112500e+10,-2.034467e+10,1.258800e+10,1.186600e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,4.316667e+08,0.0,0.0,NaN
76,NKT.CO,2023-12-31,0.117090,3.040000e+08,-7.000000e+06,-4.903333e+07,3.085000e+07,-2.380000e+08,8.880000e+08,2.620000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
77,NKT.CO,2022-12-31,0.257814,1.071000e+08,-2.500000e+06,-8.250000e+07,6.170000e+07,-1.911000e+08,2.585000e+08,2.005000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
78,NKT.CO,2021-12-31,0.240177,-4.600000e+06,0.000000e+00,-2.330000e+07,0.000000e+00,-2.134000e+08,2.005000e+08,2.392000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


### Splitting and normalization

In [35]:
test_data = df.loc[df.groupby('Ticker')['Date'].idxmax()]
training_data = df[~df.index.isin(test_data.index)]

test_data.to_csv('../data/test_data.csv', index=False)
training_data.to_csv('../data/training_data.csv', index=False)

display(test_data)
display(training_data)

,Ticker,Date,1 Year Future Change,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Capital Expenditure,End Cash Position,Beginning Cash Position,...,Non Current Accrued Expenses,Earnings From Equity Interest Net Of Tax,Depreciation Amortization Depletion Income Statement,Amortization Of Intangibles Income Statement,Preferred Stock,Non Current Note Receivables,Line Of Credit,Commercial Paper,Current Notes Payable,Preferred Stock Dividends
52,ABBN.SW,2023-12-31,0.364849,3.520000e+09,-1.258000e+09,-1.567000e+09,2.586000e+09,-7.700000e+08,3.909000e+09,4.174000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,BCKIY,2024-03-31,0.424874,1.723000e+08,-1.250000e+07,-1.310000e+07,0.000000e+00,-1.424000e+08,5.526000e+08,4.295000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,DANSKE.CO,2023-12-31,0.270911,1.401190e+11,0.000000e+00,-2.369600e+10,2.242500e+10,-1.381000e+09,3.656090e+11,2.325310e+11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,DSV.CO,2023-12-31,0.314907,1.408300e+10,-1.399700e+10,-3.270000e+08,2.120000e+08,-2.375000e+09,6.452000e+09,1.016000e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,GMAB,2023-12-31,-0.342097,7.004000e+09,-5.640000e+08,-9.100000e+07,NaN,-3.760000e+08,1.486700e+10,9.893000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,ISS.CO,2023-12-31,0.051002,2.673000e+09,0.000000e+00,0.000000e+00,0.000000e+00,-7.190000e+08,6.093000e+09,5.214000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,LLY,2023-12-31,0.357117,-3.152000e+09,-7.500000e+08,0.000000e+00,3.958500e+09,-7.392100e+09,2.818600e+09,2.067000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,MAERSK-B.CO,2023-12-31,0.064355,5.997000e+09,-3.120000e+09,-6.600000e+08,8.450000e+08,-3.646000e+09,6.683000e+09,1.003800e+10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,MATAS.CO,2024-03-31,0.153992,2.390000e+08,-2.100000e+07,-1.890000e+08,1.121000e+09,-4.060000e+08,1.310000e+08,3.700000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NETC.CO,2023-12-31,0.496028,5.522000e+08,-3.200000e+07,-3.143000e+08,1.275000e+08,-2.064000e+08,4.481000e+08,3.360000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Ticker,Date,1 Year Future Change,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Capital Expenditure,End Cash Position,Beginning Cash Position,...,Non Current Accrued Expenses,Earnings From Equity Interest Net Of Tax,Depreciation Amortization Depletion Income Statement,Amortization Of Intangibles Income Statement,Preferred Stock,Non Current Note Receivables,Line Of Credit,Commercial Paper,Current Notes Payable,Preferred Stock Dividends
1,MATAS.CO,2023-03-31,0.470333,4.240000e+08,0.000000e+00,-1.260000e+08,0.000000e+00,-2.540000e+08,3.700000e+07,2.800000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MATAS.CO,2022-03-31,-0.103056,3.271000e+08,-7.510000e+07,-7.084000e+08,7.635000e+08,-1.834000e+08,2.820000e+07,4.070000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MATAS.CO,2021-03-31,0.179800,8.036000e+08,0.000000e+00,-7.250000e+08,0.000000e+00,-1.484000e+08,4.070000e+07,1.066000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,QQ.L,2023-03-31,0.124649,9.700000e+07,-8.000000e+05,-2.579000e+08,4.811000e+08,-1.090000e+08,1.512000e+08,2.481000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,QQ.L,2022-03-31,0.067229,1.044000e+08,-8.000000e+05,0.000000e+00,0.000000e+00,-8.430000e+07,2.481000e+08,1.901000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,QQ.L,2021-03-31,-0.036492,1.021000e+08,-9.000000e+06,-8.596667e+07,1.603667e+08,-7.590000e+07,1.901000e+08,1.058000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,SAABF,2022-12-31,0.628164,2.344000e+09,0.000000e+00,-1.386000e+09,1.394000e+09,-2.310000e+09,2.869000e+09,1.701000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,SAABF,2021-12-31,0.897003,3.193000e+09,-2.460000e+08,-2.538000e+09,2.248000e+09,-2.520000e+09,1.701000e+09,2.273000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,SAABF,2020-12-31,0.041659,2.820333e+09,-2.420000e+08,-1.666333e+09,1.630667e+09,-2.789333e+09,2.233000e+09,2.281000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,BCKIY,2023-03-31,0.846253,1.362000e+08,0.000000e+00,-9.728000e+08,4.166000e+08,-1.251000e+08,4.295000e+08,7.565000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
X_train = training_data.drop(columns=['Ticker', 'Date', '1 Year Future Change'])
y_train = training_data['1 Year Future Change']
X_test = test_data.drop(columns=['Ticker', 'Date', '1 Year Future Change'])
y_test = test_data['1 Year Future Change']

display(X_train)
display(y_train)

,Free Cash Flow,Repurchase Of Capital Stock,Repayment Of Debt,Issuance Of Debt,Capital Expenditure,End Cash Position,Beginning Cash Position,Effect Of Exchange Rate Changes,Changes In Cash,Financing Cash Flow,...,Non Current Accrued Expenses,Earnings From Equity Interest Net Of Tax,Depreciation Amortization Depletion Income Statement,Amortization Of Intangibles Income Statement,Preferred Stock,Non Current Note Receivables,Line Of Credit,Commercial Paper,Current Notes Payable,Preferred Stock Dividends
1,4.240000e+08,0.000000e+00,-1.260000e+08,0.000000e+00,-2.540000e+08,3.700000e+07,2.800000e+07,0.000000e+00,9.000000e+06,-4.130000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.271000e+08,-7.510000e+07,-7.084000e+08,7.635000e+08,-1.834000e+08,2.820000e+07,4.070000e+07,2.000000e+06,-1.250000e+07,-2.914000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.036000e+08,0.000000e+00,-7.250000e+08,0.000000e+00,-1.484000e+08,4.070000e+07,1.066000e+08,2.000000e+06,-6.590000e+07,-8.400000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,9.700000e+07,-8.000000e+05,-2.579000e+08,4.811000e+08,-1.090000e+08,1.512000e+08,2.481000e+08,1.800000e+06,-9.870000e+07,1.597000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.044000e+08,-8.000000e+05,0.000000e+00,0.000000e+00,-8.430000e+07,2.481000e+08,1.901000e+08,1.300000e+06,5.670000e+07,-5.040000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.021000e+08,-9.000000e+06,-8.596667e+07,1.603667e+08,-7.590000e+07,1.901000e+08,1.058000e+08,1.150000e+07,7.280000e+07,-5.560000e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2.344000e+09,0.000000e+00,-1.386000e+09,1.394000e+09,-2.310000e+09,2.869000e+09,1.701000e+09,1.300000e+08,1.038000e+09,-1.178000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,3.193000e+09,-2.460000e+08,-2.538000e+09,2.248000e+09,-2.520000e+09,1.701000e+09,2.273000e+09,5.700000e+07,-6.290000e+08,-1.639000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2.820333e+09,-2.420000e+08,-1.666333e+09,1.630667e+09,-2.789333e+09,2.233000e+09,2.281000e+09,4.333333e+07,-9.133333e+07,-1.349000e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,1.362000e+08,0.000000e+00,-9.728000e+08,4.166000e+08,-1.251000e+08,4.295000e+08,7.565000e+08,-5.700000e+06,-3.213000e+08,-6.661000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1     0.470333
2    -0.103056
3     0.179800
5     0.124649
6     0.067229
7    -0.036492
9     0.628164
10    0.897003
11    0.041659
13    0.846253
14   -0.075000
15   -0.487180
17    0.453156
18    0.552653
19    0.959440
21   -0.251880
22    0.063221
23   -0.027054
25    0.603917
26    0.337565
27    0.660782
29    0.374533
30    0.237116
31    0.140855
33   -0.100496
34   -0.253455
35    0.767016
37   -0.115871
38    0.178400
39    0.185958
41    0.071625
42   -0.279177
43    0.508148
45    0.081298
46   -0.055432
47   -0.056061
49   -0.238352
50   -0.582399
51    0.133788
53    0.348640
54   -0.138713
55    0.497111
57    0.245785
58   -0.464103
59    0.368076
61    0.672438
62   -0.518241
63    0.114124
65    0.048279
66    2.879940
67    0.148889
69    0.014749
70   -0.061772
71    0.390665
73    0.502751
74    0.295327
75    0.756239
77    0.257814
78    0.240177
79    0.163717
Name: 1 Year Future Change, dtype: float64

## Model Training

In [37]:
model = MLPClassifier(
    solver='adam',
    alpha=1e-4,
    hidden_layer_sizes=(50, 50, 50),
    shuffle=False,
    random_state=42,
    tol=0.0001,
    verbose=True,
    max_iter=50,
    batch_size=1024,
    learning_rate='adaptive',
    n_iter_no_change=4
)
model.fit(X_train, y_train)

ValueError: Input X contains NaN.
MLPClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

## Testing and benchmarking

## Predictions